In [ ]:
#!/usr/bin/env python3.6
import string
import pickle
from numpy import zeros


def clean_text(text: str):
    text = text.lower()
    wordss = text.split()
    table = str.maketrans('', '', string.punctuation)
    wordss = [word.translate(table) for word in wordss]
    ss = [word.lower() for word in wordss if word.isalpha()]
    return ss


def CleanCat(x: str) -> str:
    return x.replace(",", "").replace("\"", "")


def oneHot(data, cats=None):
    categories = cats if cats is not None else list(set(cats))
    oneHotMatrix = zeros(shape=(len(data), len(categories)))
    oneHotDict = {}
    for i in range(len(categories)):
        zer = zeros((len(categories, )))
        zer[i] = 1
        oneHotDict[categories[i]] = zer
    for j in range(len(data)):
        oneHotMatrix[j] = oneHotDict[data[j]]
    return oneHotMatrix


if __name__ == "__main__":
    test = "test,,,,,,,,,,,,,,,,"
    print(CleanCat(test))
    with open('drive/MyDrive/colabordn/dosyalar/saved/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    print(len(tokenizer.index_word))


test
13490


In [ ]:
#!/usr/bin/env python3.6
import pickle
import tensorflow as tf


class Detector:
    def __init__(self, vocSize):
        self.build(vocSize)

    def predict(self, testx):
        pass

    def trainOnData(self, x, y, epochs, batchSize):
        for epoch in range(1, epochs + 1):
            for batch in range(x.shape[0] // batchSize):
                x1 = x[batch * batchSize:min(len(x), (batch + 1) * batchSize)]
                y1 = y[batch * batchSize:min(len(y), (batch + 1) * batchSize)]
                with tf.GradientTape() as tape:
                    pred = self.model(x1)
                    lossn = self.loss(y1, pred)
                    print(lossn)
            grads = tape.gradient(lossn, self.model.trainable_variables)
            self.opt.apply_gradients(zip(grads, self.model.trainable_variables))
        self.model.fit(x, y, batch_size=batchSize, epochs=epochs)
        self.model.save("saved/model.h5")

    def build(self, vocSize, modelBuild=True):
        with open('drive/MyDrive/colabordn/dosyalar/saved/tokenizer.pickle', 'rb') as file:
            self.tok = pickle.load(file)
        if modelBuild:
            self.model = tf.keras.Sequential([
                tf.keras.layers.Embedding(vocSize, 50, input_length=302),
                tf.keras.layers.GRU(50, return_sequences=True),
                tf.keras.layers.GlobalAveragePooling1D(),
                tf.keras.layers.Dense(5, activation="softmax")
            ])
            self.opt = tf.keras.optimizers.Adam(0.001)
            self.loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
            self.model.compile(self.opt, self.loss, [tf.keras.metrics.Accuracy(), tf.keras.metrics.Recall(),
                                                     tf.keras.metrics.BinaryAccuracy()])


In [ ]:
#!/usr/bin/env python3.6
import pandas as pd
import numpy as np
import tensorflow as tf

"""from sklearn.preprocessing import OneHotEncoder,LabelEncoder"""
import pickle
import warnings

warnings.filterwarnings("ignore")
data = pd.read_csv("drive/MyDrive/colabordn/dosyalar/teknofest_train_final.csv", encoding="utf8", lineterminator="\r", on_bad_lines="skip", sep="|")
data = data.dropna()
with tf.device('/GPU:0'):
  tok = tf.keras.preprocessing.text.Tokenizer(20000)
  datax = []
  datay = np.squeeze(data[["target,,,,,"]].to_numpy())
  for i in range(len(datay)):
      datay[i] = CleanCat(datay[i])
  print(np.unique(datay))
  # one=OneHotEncoder()
  """le=LabelEncoder()
  datay=np.reshape(le.fit_transform(datay),(-1,1))"""
  # datay=np.array(one.fit_transform(datay))
  datay = oneHot(datay, np.unique(datay))
  print(datay)
  MAX = 302
  for i in data.text:
      datax.append(clean_text(i))
  tok.fit_on_texts(datax)
  datax = tok.texts_to_sequences(datax)
  with open("drive/MyDrive/colabordn/dosyalar/saved/tokenizer.pickle", "wb") as file:
      pickle.dump(tok, file, pickle.HIGHEST_PROTOCOL)
      pickle.dump(tok, file, pickle.HIGHEST_PROTOCOL)
  datax = tf.keras.utils.pad_sequences(datax, MAX)
  det = Detector(20000)
  det.trainOnData(datax, datay, 50, 128)


Görüntülenen çıkış son 5000 satıra kısaltıldı.
tf.Tensor(0.6901308, shape=(), dtype=float32)
tf.Tensor(0.68942916, shape=(), dtype=float32)
tf.Tensor(0.69017863, shape=(), dtype=float32)
tf.Tensor(0.68972015, shape=(), dtype=float32)
tf.Tensor(0.6895031, shape=(), dtype=float32)
tf.Tensor(0.6905183, shape=(), dtype=float32)
tf.Tensor(0.6898515, shape=(), dtype=float32)
tf.Tensor(0.6898726, shape=(), dtype=float32)
tf.Tensor(0.68945, shape=(), dtype=float32)
tf.Tensor(0.6902991, shape=(), dtype=float32)
tf.Tensor(0.6898174, shape=(), dtype=float32)
tf.Tensor(0.68946904, shape=(), dtype=float32)
tf.Tensor(0.68922335, shape=(), dtype=float32)
tf.Tensor(0.6897687, shape=(), dtype=float32)
tf.Tensor(0.690022, shape=(), dtype=float32)
tf.Tensor(0.6897843, shape=(), dtype=float32)
tf.Tensor(0.6903095, shape=(), dtype=float32)
tf.Tensor(0.68990207, shape=(), dtype=float32)
tf.Tensor(0.6896614, shape=(), dtype=float32)
tf.Tensor(0.6904646, shape=(), dtype=float32)
tf.Tensor(0.68988156, shape=()